In [ ]:
import ee
import time
import geopandas as gpd
import os
import json
import pandas as pd
import logging
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2

from retry import retry
from dotenv import load_dotenv
import argparse, sys

current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
load_dotenv()

In [ ]:
##################################
# Provide a log to trace the process
##################################
LOG_FILE=os.environ['LOG_FILE_FLII']
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.getLogger().addHandler(logging.StreamHandler())
_str_decorator = "=" * 20
logging.info(f"\n{_str_decorator} BEGINNING LOG {_str_decorator}")


##################################
# Initialize Earth Engine
##################################
SERVICE_ACCOUNT = os.environ['SERVICE_ACCOUNT']
KEY_IAM = os.environ["KEY_IAM"]
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_IAM)
# Using high-volume EE endpoints to make requests simultaneously as recommended by Gorelick in this blogpost https://gorelick.medium.com/fast-er-downloads-a2abd512aa26
ee.Initialize(credentials, opt_url='https://earthengine-highvolume.googleapis.com')

##################################
### SET YOUR INITIAL PARAMETER HERE
##################################
dbname = os.environ['DB_GIS_NAME']
host = os.environ['DB_GIS_HOST']
port = os.environ['DB_GIS_PORT']
passwd = os.environ['DB_GIS_PASS']
user = os.environ['DB_GIS_USER']
db_connection_uri = "postgresql://"+user+":"+passwd+"@"+host+":"+port+"/"+dbname+""
engine = create_engine(db_connection_uri)
psy_conn = psycopg2.connect(
    host=host,
    port=port,
    database=dbname,
    user=user,
    password=passwd
)
cursor = psy_conn.cursor()

def array_calc(number):
    return ee.Number(0.75).pow(ee.Number(number))

def array_calc_defaun(number):
    return ee.Number(19).subtract(ee.Number(number))

class FLII(object):
    
    def __init__(self, year):
        self.drive_folder = os.environ['DRIVE_FOLDER_FLII']
        # self.output_file = output_file
        # self.output_filename = output_filename
        self.year = year
        self.infrastructure = ee.Image('projects/wcs-forest-second-backup/assets/osm_22_rast_300/new_infra_22')
        self.deforestation = ee.Image('users/aduncan/flii2_defor_direct/flii2v6_defor_dropLTE6_22')
        self.crop = ee.Image('users/aduncan/flii2_crop/flii2_crop_2019') # why do you need to refer this asset? What's the purpose?
        self.connectivity = ee.Image('users/aduncan/osm_earth/flii2v6_total_connectivity_PRE2022') # what kind of feature is this?
        self.boreal_connectivity = ee.Image('users/aduncan/osm_earth/total_connectivity_original_borealfixed') # is it boreal forest?
        self.start_date = '2017-03-01'
        self.end_date = '2017-03-31'
        self.scale = 300
        # This dataset contains maps of the location and temporal distribution of surface water from 1984 to 2015 and provides statistics on the extent and change of those water surfaces.
        # These data were generated using 3,066,102 scenes from Landsat 5, 7, and 8 acquired between 16 March 1984 and 10 October 2015. Each pixel was individually classified into water / non-water using an expert system and the results were collated into a monthly history for the entire time period and two epochs (1984-1999, 2000-2015) for change detection.
        self.water_extent = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('occurrence').lte(75).unmask(1)
        self.ocean = ee.Image('users/aduncan/cci/ESACCI-LC-L4-WB-Ocean-Map-150m-P13Y-2000-v40')
        self.direct_sanity = ee.Image('users/aduncan/flii2_direct/total_direct_pressure_2017')
        self.indirect_sanity = ee.Image('users/yourecoveredinbees/flii2_ephemeral/total_indirect_pressure_2017')
        # self.start_date = self.get_start_end()[0]
        # self.end_date = self.get_start_end()[1]
        self.aoi = None
        self.infra_hawths = self.hawths(self.infrastructure, 0.254)
        self.crop_hawths = self.hawths(self.crop, 2.069)
        self.defo_hawths = self.hawths(self.deforestation, 8.535)
        self.total_direct_pressure = self.infra_hawths.add(self.crop_hawths).add(self.defo_hawths).updateMask(self.water_extent).updateMask(self.ocean).unmask(0)
        self.crs = 'EPSG:4326'
        self.total_indirect_pressure = self.total_direct_pressure.reduceNeighborhood(
                    reducer=ee.Reducer.mean(),
                    kernel=self.kernelIndirect()).multiply(2).reproject(crs=self.crs,scale=self.scale)
        
        # I have to unmask the direct pressure, perform the calculation, then mask again because of the changing resolution
        self.defaun_pressure = self.total_direct_pressure.reduceNeighborhood(
                    reducer=ee.Reducer.sum(),
                    kernel=self.kernelDefaun()).reproject(crs=self.crs,scale=600)
        self.final_defaun_pressure = self.defaun_pressure.resample().reproject(crs=self.crs,scale=self.scale).multiply(0.25).where(self.defaun_pressure.gte(0.1),0.1)
        self.total_pressure_raw = self.total_direct_pressure.add(self.total_indirect_pressure).add(self.defaun_pressure).updateMask(self.water_extent).updateMask(self.ocean) 
        self.ratio = self.connectivity.divide(self.boreal_connectivity.unmask(0.001).where(self.boreal_connectivity.eq(0),0.001))
        self.final_ratio = self.ratio.where(self.ratio.gt(1),1)
        
    def hawths(self, image, exp_gamma):
        return ee.Image(1).subtract(image.multiply(-1).multiply(ee.Number(exp_gamma)).exp())
    
    def kernelIndirect(self):
        _weight1 = [999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,19,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999]
        _weight2 = [999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,18.9736659610103,18.6815416922694,18.4390889145858,18.2482875908947,18.1107702762748,18.0277563773199,18,18.0277563773199,18.1107702762748,18.2482875908947,18.4390889145858,18.6815416922694,18.9736659610103,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999]
        _weight3 = [999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,18.7882942280559,18.3847763108502,18.0277563773199,17.7200451466694,17.464249196573,17.2626765016321,17.1172427686237,17.0293863659264,17,17.0293863659264,17.1172427686237,17.2626765016321,17.464249196573,17.7200451466694,18.0277563773199,18.3847763108502,18.7882942280559,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999]
        _weight4 = [999999,999999,999999,999999,999999,999999,999999,999999,999999,18.8679622641132,18.3575597506858,17.8885438199983,17.464249196573,17.0880074906351,16.7630546142402,16.4924225024706,16.2788205960997,16.1245154965971,16.0312195418814,16,16.0312195418814,16.1245154965971,16.2788205960997,16.4924225024706,16.7630546142402,17.0880074906351,17.464249196573,17.8885438199983,18.3575597506858,18.8679622641132,999999,999999,999999,999999,999999,999999,999999,999999,999999]
        _weight5 = [999999,999999,999999,999999,999999,999999,999999,999999,18.6010752377383,18.0277563773199,17.4928556845359,17,16.5529453572468,16.1554944214035,15.8113883008419,15.52417469626,15.2970585407784,15.1327459504216,15.0332963783729,15,15.0332963783729,15.1327459504216,15.2970585407784,15.52417469626,15.8113883008419,16.1554944214035,16.5529453572468,17,17.4928556845359,18.0277563773199,18.6010752377383,999999,999999,999999,999999,999999,999999,999999,999999]
        _weight6 = [999999,999999,999999,999999,999999,999999,999999,18.4390889145858,17.8044938147649,17.2046505340853,16.6433169770932,16.1245154965971,15.6524758424985,15.2315462117278,14.8660687473185,14.560219778561,14.3178210632764,14.142135623731,14.0356688476182,14,14.0356688476182,14.142135623731,14.3178210632764,14.560219778561,14.8660687473185,15.2315462117278,15.6524758424985,16.1245154965971,16.6433169770932,17.2046505340853,17.8044938147649,18.4390889145858,999999,999999,999999,999999,999999,999999,999999]
        _weight7 = [999999,999999,999999,999999,999999,999999,18.3847763108502,17.6918060129541,17.0293863659264,16.4012194668567,15.8113883008419,15.2643375224737,14.7648230602334,14.3178210632764,13.9283882771841,13.6014705087354,13.3416640641263,13.1529464379659,13.0384048104053,13,13.0384048104053,13.1529464379659,13.3416640641263,13.6014705087354,13.9283882771841,14.3178210632764,14.7648230602334,15.2643375224737,15.8113883008419,16.4012194668567,17.0293863659264,17.6918060129541,18.3847763108502,999999,999999,999999,999999,999999,999999]
        _weight8 = [999999,999999,999999,999999,999999,18.4390889145858,17.6918060129541,16.9705627484771,16.2788205960997,15.6204993518133,15,14.422205101856,13.8924439894498,13.4164078649987,13,12.6491106406735,12.369316876853,12.1655250605964,12.0415945787923,12,12.0415945787923,12.1655250605964,12.369316876853,12.6491106406735,13,13.4164078649987,13.8924439894498,14.422205101856,15,15.6204993518133,16.2788205960997,16.9705627484771,17.6918060129541,18.4390889145858,999999,999999,999999,999999,999999]
        _weight9 = [999999,999999,999999,999999,18.6010752377383,17.8044938147649,17.0293863659264,16.2788205960997,15.556349186104,14.8660687473185,14.2126704035519,13.6014705087354,13.0384048104053,12.5299640861417,12.0830459735946,11.7046999107196,11.4017542509914,11.180339887499,11.0453610171873,11,11.0453610171873,11.180339887499,11.4017542509914,11.7046999107196,12.0830459735946,12.5299640861417,13.0384048104053,13.6014705087354,14.2126704035519,14.8660687473185,15.556349186104,16.2788205960997,17.0293863659264,17.8044938147649,18.6010752377383,999999,999999,999999,999999]
        _weight10 = [999999,999999,999999,18.8679622641132,18.0277563773199,17.2046505340853,16.4012194668567,15.6204993518133,14.8660687473185,14.142135623731,13.4536240470737,12.8062484748657,12.2065556157337,11.6619037896906,11.180339887499,10.770329614269,10.4403065089106,10.1980390271856,10.0498756211209,10,10.0498756211209,10.1980390271856,10.4403065089106,10.770329614269,11.180339887499,11.6619037896906,12.2065556157337,12.8062484748657,13.4536240470737,14.142135623731,14.8660687473185,15.6204993518133,16.4012194668567,17.2046505340853,18.0277563773199,18.8679622641132,999999,999999,999999]
        _weight11 = [999999,999999,999999,18.3575597506858,17.4928556845359,16.6433169770932,15.8113883008419,15,14.2126704035519,13.4536240470737,12.7279220613579,12.0415945787923,11.4017542509914,10.816653826392,10.295630140987,9.8488578017961,9.48683298050514,9.21954445729289,9.05538513813742,9,9.05538513813742,9.21954445729289,9.48683298050514,9.8488578017961,10.295630140987,10.816653826392,11.4017542509914,12.0415945787923,12.7279220613579,13.4536240470737,14.2126704035519,15,15.8113883008419,16.6433169770932,17.4928556845359,18.3575597506858,999999,999999,999999]
        _weight12 = [999999,999999,18.7882942280559,17.8885438199983,17,16.1245154965971,15.2643375224737,14.422205101856,13.6014705087354,12.8062484748657,12.0415945787923,11.3137084989848,10.6301458127347,10,9.4339811320566,8.94427190999916,8.54400374531753,8.24621125123532,8.06225774829855,8,8.06225774829855,8.24621125123532,8.54400374531753,8.94427190999916,9.4339811320566,10,10.6301458127347,11.3137084989848,12.0415945787923,12.8062484748657,13.6014705087354,14.422205101856,15.2643375224737,16.1245154965971,17,17.8885438199983,18.7882942280559,999999,999999]
        _weight13 = [999999,999999,18.3847763108502,17.464249196573,16.5529453572468,15.6524758424985,14.7648230602334,13.8924439894498,13.0384048104053,12.2065556157337,11.4017542509914,10.6301458127347,9.89949493661167,9.21954445729289,8.60232526704263,8.06225774829855,7.61577310586391,7.28010988928052,7.07106781186548,7,7.07106781186548,7.28010988928052,7.61577310586391,8.06225774829855,8.60232526704263,9.21954445729289,9.89949493661167,10.6301458127347,11.4017542509914,12.2065556157337,13.0384048104053,13.8924439894498,14.7648230602334,15.6524758424985,16.5529453572468,17.464249196573,18.3847763108502,999999,999999]
        _weight14 = [999999,18.9736659610103,18.0277563773199,17.0880074906351,16.1554944214035,15.2315462117278,14.3178210632764,13.4164078649987,12.5299640861417,11.6619037896906,10.816653826392,10,9.21954445729289,8.48528137423857,7.81024967590665,7.21110255092798,6.70820393249937,6.32455532033676,6.08276253029822,6,6.08276253029822,6.32455532033676,6.70820393249937,7.21110255092798,7.81024967590665,8.48528137423857,9.21954445729289,10,10.816653826392,11.6619037896906,12.5299640861417,13.4164078649987,14.3178210632764,15.2315462117278,16.1554944214035,17.0880074906351,18.0277563773199,18.9736659610103,999999]
        _weight15 = [999999,18.6815416922694,17.7200451466694,16.7630546142402,15.8113883008419,14.8660687473185,13.9283882771841,13,12.0830459735946,11.180339887499,10.295630140987,9.4339811320566,8.60232526704263,7.81024967590665,7.07106781186548,6.40312423743285,5.8309518948453,5.3851648071345,5.09901951359278,5,5.09901951359278,5.3851648071345,5.8309518948453,6.40312423743285,7.07106781186548,7.81024967590665,8.60232526704263,9.4339811320566,10.295630140987,11.180339887499,12.0830459735946,13,13.9283882771841,14.8660687473185,15.8113883008419,16.7630546142402,17.7200451466694,18.6815416922694,999999]
        _weight16 = [999999,18.4390889145858,17.464249196573,16.4924225024706,15.52417469626,14.560219778561,13.6014705087354,12.6491106406735,11.7046999107196,10.770329614269,9.8488578017961,8.94427190999916,8.06225774829855,7.21110255092798,6.40312423743285,5.65685424949238,5,4.47213595499958,4.12310562561766,4,4.12310562561766,4.47213595499958,5,5.65685424949238,6.40312423743285,7.21110255092798,8.06225774829855,8.94427190999916,9.8488578017961,10.770329614269,11.7046999107196,12.6491106406735,13.6014705087354,14.560219778561,15.52417469626,16.4924225024706,17.464249196573,18.4390889145858,999999]
        _weight17 = [999999,18.2482875908947,17.2626765016321,16.2788205960997,15.2970585407784,14.3178210632764,13.3416640641263,12.369316876853,11.4017542509914,10.4403065089106,9.48683298050514,8.54400374531753,7.61577310586391,6.70820393249937,5.8309518948453,5,4.24264068711929,3.60555127546399,3.16227766016838,3,3.16227766016838,3.60555127546399,4.24264068711929,5,5.8309518948453,6.70820393249937,7.61577310586391,8.54400374531753,9.48683298050514,10.4403065089106,11.4017542509914,12.369316876853,13.3416640641263,14.3178210632764,15.2970585407784,16.2788205960997,17.2626765016321,18.2482875908947,999999]
        _weight18 = [999999,18.1107702762748,17.1172427686237,16.1245154965971,15.1327459504216,14.142135623731,13.1529464379659,12.1655250605964,11.180339887499,10.1980390271856,9.21954445729289,8.24621125123532,7.28010988928052,6.32455532033676,5.3851648071345,4.47213595499958,3.60555127546399,2.82842712474619,2.23606797749979,2,2.23606797749979,2.82842712474619,3.60555127546399,4.47213595499958,5.3851648071345,6.32455532033676,7.28010988928052,8.24621125123532,9.21954445729289,10.1980390271856,11.180339887499,12.1655250605964,13.1529464379659,14.142135623731,15.1327459504216,16.1245154965971,17.1172427686237,18.1107702762748,999999]
        _weight19 = [999999,18.0277563773199,17.0293863659264,16.0312195418814,15.0332963783729,14.0356688476182,13.0384048104053,12.0415945787923,11.0453610171873,10.0498756211209,9.05538513813742,8.06225774829855,7.07106781186548,6.08276253029822,5.09901951359278,4.12310562561766,3.16227766016838,2.23606797749979,1.4142135623731,1,1.4142135623731,2.23606797749979,3.16227766016838,4.12310562561766,5.09901951359278,6.08276253029822,7.07106781186548,8.06225774829855,9.05538513813742,10.0498756211209,11.0453610171873,12.0415945787923,13.0384048104053,14.0356688476182,15.0332963783729,16.0312195418814,17.0293863659264,18.0277563773199,999999]
        _weight20 = [19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 999999, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
        
        weight1 = [array_calc(number) for number in _weight1]
        weight2 = [array_calc(number) for number in _weight2]
        weight3 = [array_calc(number) for number in _weight3]
        weight4 = [array_calc(number) for number in _weight4]
        weight5 = [array_calc(number) for number in _weight5]
        weight6 = [array_calc(number) for number in _weight6]
        weight7 = [array_calc(number) for number in _weight7]
        weight8 = [array_calc(number) for number in _weight8]
        weight9 = [array_calc(number) for number in _weight9]
        weight10 = [array_calc(number) for number in _weight10]
        weight11 = [array_calc(number) for number in _weight11]
        weight12 = [array_calc(number) for number in _weight12]
        weight13 = [array_calc(number) for number in _weight13]
        weight14 = [array_calc(number) for number in _weight14]
        weight15 = [array_calc(number) for number in _weight15]
        weight16 = [array_calc(number) for number in _weight16]
        weight17 = [array_calc(number) for number in _weight17]
        weight18 = [array_calc(number) for number in _weight18]
        weight19 = [array_calc(number) for number in _weight19]
        weight20 = [array_calc(number) for number in _weight20]

        # Assemble a list of lists: the 39x39 kernel weights as a 2-D matrix.
        kernellists = [weight1, weight2, weight3, weight4, weight5, weight6, weight7, weight8, weight9, weight10, weight11, weight12, weight13, weight14,
                        weight15, weight16, weight17, weight18, weight19, weight20, weight19, weight18, weight17, weight16, weight15,  weight14, weight13, weight12, weight11, weight10, weight9, weight8,
                        weight7, weight6, weight5, weight4, weight3, weight2, weight1]
        # Create the kernel from the weights.
        fixedkernel = ee.Kernel.fixed(39, 39, kernellists, -20, -20, True)
        
        return fixedkernel
    
    def exportToAsset(self, image, description, assetId):
        task = ee.batch.Export.image.toAsset(
            image=image, 
            description=description,
            assetId=assetId, 
            region=self.aoi,
            scale=self.scale,
            crs=self.crs,
            maxPixels=1e13)
        
        task.start()
        while task.active():
            print(f"Waiting on (id: {task.id})")
            time.sleep(30)
            
    def kernelDefaun(self):
            
        _weight_defaun1 = [19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19]
        _weight_defaun2 = [19,19,19,19,19,19,19,19,19,19,19,19,19,18.9736659610103,18.6815416922694,18.4390889145858,18.2482875908947,18.1107702762748,18.0277563773199,18,18.0277563773199,18.1107702762748,18.2482875908947,18.4390889145858,18.6815416922694,18.9736659610103,19,19,19,19,19,19,19,19,19,19,19,19,19]
        _weight_defaun3 = [19,19,19,19,19,19,19,19,19,19,19,18.7882942280559,18.3847763108502,18.0277563773199,17.7200451466694,17.464249196573,17.2626765016321,17.1172427686237,17.0293863659264,17,17.0293863659264,17.1172427686237,17.2626765016321,17.464249196573,17.7200451466694,18.0277563773199,18.3847763108502,18.7882942280559,19,19,19,19,19,19,19,19,19,19,19]
        _weight_defaun4 = [19,19,19,19,19,19,19,19,19,18.8679622641132,18.3575597506858,17.8885438199983,17.464249196573,17.0880074906351,16.7630546142402,16.4924225024706,16.2788205960997,16.1245154965971,16.0312195418814,16,16.0312195418814,16.1245154965971,16.2788205960997,16.4924225024706,16.7630546142402,17.0880074906351,17.464249196573,17.8885438199983,18.3575597506858,18.8679622641132,19,19,19,19,19,19,19,19,19]
        _weight_defaun5 = [19,19,19,19,19,19,19,19,18.6010752377383,18.0277563773199,17.4928556845359,17,16.5529453572468,16.1554944214035,15.8113883008419,15.52417469626,15.2970585407784,15.1327459504216,15.0332963783729,15,15.0332963783729,15.1327459504216,15.2970585407784,15.52417469626,15.8113883008419,16.1554944214035,16.5529453572468,17,17.4928556845359,18.0277563773199,18.6010752377383,19,19,19,19,19,19,19,19]
        _weight_defaun6 = [19,19,19,19,19,19,19,18.4390889145858,17.8044938147649,17.2046505340853,16.6433169770932,16.1245154965971,15.6524758424985,15.2315462117278,14.8660687473185,14.560219778561,14.3178210632764,14.142135623731,14.0356688476182,14,14.0356688476182,14.142135623731,14.3178210632764,14.560219778561,14.8660687473185,15.2315462117278,15.6524758424985,16.1245154965971,16.6433169770932,17.2046505340853,17.8044938147649,18.4390889145858,19,19,19,19,19,19,19]
        _weight_defaun7 = [19,19,19,19,19,19,18.3847763108502,17.6918060129541,17.0293863659264,16.4012194668567,15.8113883008419,15.2643375224737,14.7648230602334,14.3178210632764,13.9283882771841,13.6014705087354,13.3416640641263,13.1529464379659,13.0384048104053,13,13.0384048104053,13.1529464379659,13.3416640641263,13.6014705087354,13.9283882771841,14.3178210632764,14.7648230602334,15.2643375224737,15.8113883008419,16.4012194668567,17.0293863659264,17.6918060129541,18.3847763108502,19,19,19,19,19,19]
        _weight_defaun8 = [19,19,19,19,19,18.4390889145858,17.6918060129541,16.9705627484771,16.2788205960997,15.6204993518133,15,14.422205101856,13.8924439894498,13.4164078649987,13,12.6491106406735,12.369316876853,12.1655250605964,12.0415945787923,12,12.0415945787923,12.1655250605964,12.369316876853,12.6491106406735,13,13.4164078649987,13.8924439894498,14.422205101856,15,15.6204993518133,16.2788205960997,16.9705627484771,17.6918060129541,18.4390889145858,19,19,19,19,19]
        _weight_defaun9 = [19,19,19,19,18.6010752377383,17.8044938147649,17.0293863659264,16.2788205960997,15.556349186104,14.8660687473185,14.2126704035519,13.6014705087354,13.0384048104053,12.5299640861417,12.0830459735946,11.7046999107196,11.4017542509914,11.180339887499,11.0453610171873,11,11.0453610171873,11.180339887499,11.4017542509914,11.7046999107196,12.0830459735946,12.5299640861417,13.0384048104053,13.6014705087354,14.2126704035519,14.8660687473185,15.556349186104,16.2788205960997,17.0293863659264,17.8044938147649,18.6010752377383,19,19,19,19]
        _weight_defaun10 = [19,19,19,18.8679622641132,18.0277563773199,17.2046505340853,16.4012194668567,15.6204993518133,14.8660687473185,14.142135623731,13.4536240470737,12.8062484748657,12.2065556157337,11.6619037896906,11.180339887499,10.770329614269,10.4403065089106,10.1980390271856,10.0498756211209,10,10.0498756211209,10.1980390271856,10.4403065089106,10.770329614269,11.180339887499,11.6619037896906,12.2065556157337,12.8062484748657,13.4536240470737,14.142135623731,14.8660687473185,15.6204993518133,16.4012194668567,17.2046505340853,18.0277563773199,18.8679622641132,19,19,19]
        _weight_defaun11 = [19,19,19,18.3575597506858,17.4928556845359,16.6433169770932,15.8113883008419,15,14.2126704035519,13.4536240470737,12.7279220613579,12.0415945787923,11.4017542509914,10.816653826392,10.295630140987,9.8488578017961,9.48683298050514,9.21954445729289,9.05538513813742,9,9.05538513813742,9.21954445729289,9.48683298050514,9.8488578017961,10.295630140987,10.816653826392,11.4017542509914,12.0415945787923,12.7279220613579,13.4536240470737,14.2126704035519,15,15.8113883008419,16.6433169770932,17.4928556845359,18.3575597506858,19,19,19]
        _weight_defaun12 = [19,19,18.7882942280559,17.8885438199983,17,16.1245154965971,15.2643375224737,14.422205101856,13.6014705087354,12.8062484748657,12.0415945787923,11.3137084989848,10.6301458127347,10,9.4339811320566,8.94427190999916,8.54400374531753,8.24621125123532,8.06225774829855,8,8.06225774829855,8.24621125123532,8.54400374531753,8.94427190999916,9.4339811320566,10,10.6301458127347,11.3137084989848,12.0415945787923,12.8062484748657,13.6014705087354,14.422205101856,15.2643375224737,16.1245154965971,17,17.8885438199983,18.7882942280559,19,19]
        _weight_defaun13 = [19,19,18.3847763108502,17.464249196573,16.5529453572468,15.6524758424985,14.7648230602334,13.8924439894498,13.0384048104053,12.2065556157337,11.4017542509914,10.6301458127347,9.89949493661167,9.21954445729289,8.60232526704263,8.06225774829855,7.61577310586391,7.28010988928052,7.07106781186548,7,7.07106781186548,7.28010988928052,7.61577310586391,8.06225774829855,8.60232526704263,9.21954445729289,9.89949493661167,10.6301458127347,11.4017542509914,12.2065556157337,13.0384048104053,13.8924439894498,14.7648230602334,15.6524758424985,16.5529453572468,17.464249196573,18.3847763108502,19,19]
        _weight_defaun14 = [19,18.9736659610103,18.0277563773199,17.0880074906351,16.1554944214035,15.2315462117278,14.3178210632764,13.4164078649987,12.5299640861417,11.6619037896906,10.816653826392,10,9.21954445729289,8.48528137423857,7.81024967590665,7.21110255092798,6.70820393249937,6.32455532033676,6.08276253029822,6,6.08276253029822,6.32455532033676,6.70820393249937,7.21110255092798,7.81024967590665,8.48528137423857,9.21954445729289,10,10.816653826392,11.6619037896906,12.5299640861417,13.4164078649987,14.3178210632764,15.2315462117278,16.1554944214035,17.0880074906351,18.0277563773199,18.9736659610103,19]
        _weight_defaun15 = [19,18.6815416922694,17.7200451466694,16.7630546142402,15.8113883008419,14.8660687473185,13.9283882771841,13,12.0830459735946,11.180339887499,10.295630140987,9.4339811320566,8.60232526704263,7.81024967590665,7.07106781186548,6.40312423743285,5.8309518948453,5.3851648071345,5.09901951359278,5,5.09901951359278,5.3851648071345,5.8309518948453,6.40312423743285,7.07106781186548,7.81024967590665,8.60232526704263,9.4339811320566,10.295630140987,11.180339887499,12.0830459735946,13,13.9283882771841,14.8660687473185,15.8113883008419,16.7630546142402,17.7200451466694,18.6815416922694,19]
        _weight_defaun16 = [19,18.4390889145858,17.464249196573,16.4924225024706,15.52417469626,14.560219778561,13.6014705087354,12.6491106406735,11.7046999107196,10.770329614269,9.8488578017961,8.94427190999916,8.06225774829855,7.21110255092798,6.40312423743285,5.65685424949238,5,4.47213595499958,4.12310562561766,4,4.12310562561766,4.47213595499958,5,5.65685424949238,6.40312423743285,7.21110255092798,8.06225774829855,8.94427190999916,9.8488578017961,10.770329614269,11.7046999107196,12.6491106406735,13.6014705087354,14.560219778561,15.52417469626,16.4924225024706,17.464249196573,18.4390889145858,19]
        _weight_defaun17 = [19,18.2482875908947,17.2626765016321,16.2788205960997,15.2970585407784,14.3178210632764,13.3416640641263,12.369316876853,11.4017542509914,10.4403065089106,9.48683298050514,8.54400374531753,7.61577310586391,6.70820393249937,5.8309518948453,5,4.24264068711929,3.60555127546399,3.16227766016838,3,3.16227766016838,3.60555127546399,4.24264068711929,5,5.8309518948453,6.70820393249937,7.61577310586391,8.54400374531753,9.48683298050514,10.4403065089106,11.4017542509914,12.369316876853,13.3416640641263,14.3178210632764,15.2970585407784,16.2788205960997,17.2626765016321,18.2482875908947,19]
        _weight_defaun18 = [19,18.1107702762748,17.1172427686237,16.1245154965971,15.1327459504216,14.142135623731,13.1529464379659,12.1655250605964,11.180339887499,10.1980390271856,9.21954445729289,8.24621125123532,7.28010988928052,6.32455532033676,5.3851648071345,4.47213595499958,3.60555127546399,2.82842712474619,2.23606797749979,2,2.23606797749979,2.82842712474619,3.60555127546399,4.47213595499958,5.3851648071345,6.32455532033676,7.28010988928052,8.24621125123532,9.21954445729289,10.1980390271856,11.180339887499,12.1655250605964,13.1529464379659,14.142135623731,15.1327459504216,16.1245154965971,17.1172427686237,18.1107702762748,19]
        _weight_defaun19 = [19,18.0277563773199,17.0293863659264,16.0312195418814,15.0332963783729,14.0356688476182,13.0384048104053,12.0415945787923,11.0453610171873,10.0498756211209,9.05538513813742,8.06225774829855,7.07106781186548,6.08276253029822,5.09901951359278,4.12310562561766,3.16227766016838,2.23606797749979,1.4142135623731,1,1.4142135623731,2.23606797749979,3.16227766016838,4.12310562561766,5.09901951359278,6.08276253029822,7.07106781186548,8.06225774829855,9.05538513813742,10.0498756211209,11.0453610171873,12.0415945787923,13.0384048104053,14.0356688476182,15.0332963783729,16.0312195418814,17.0293863659264,18.0277563773199,19]
        _weight_defaun20 = [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,19,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

        weight1 = [array_calc_defaun(number) for number in _weight_defaun1]
        weight2 = [array_calc_defaun(number) for number in _weight_defaun2]
        weight3 = [array_calc_defaun(number) for number in _weight_defaun3]
        weight4 = [array_calc_defaun(number) for number in _weight_defaun4]
        weight5 = [array_calc_defaun(number) for number in _weight_defaun5]
        weight6 = [array_calc_defaun(number) for number in _weight_defaun6]
        weight7 = [array_calc_defaun(number) for number in _weight_defaun7]
        weight8 = [array_calc_defaun(number) for number in _weight_defaun8]
        weight9 = [array_calc_defaun(number) for number in _weight_defaun9]
        weight10 = [array_calc_defaun(number) for number in _weight_defaun10]
        weight11 = [array_calc_defaun(number) for number in _weight_defaun11]
        weight12 = [array_calc_defaun(number) for number in _weight_defaun12]
        weight13 = [array_calc_defaun(number) for number in _weight_defaun13]
        weight14 = [array_calc_defaun(number) for number in _weight_defaun14]
        weight15 = [array_calc_defaun(number) for number in _weight_defaun15]
        weight16 = [array_calc_defaun(number) for number in _weight_defaun16]
        weight17 = [array_calc_defaun(number) for number in _weight_defaun17]
        weight18 = [array_calc_defaun(number) for number in _weight_defaun18]
        weight19 = [array_calc_defaun(number) for number in _weight_defaun19]
        weight20 = [array_calc_defaun(number) for number in _weight_defaun20]
        
        # Assemble a list of lists: the 39x39 kernel weights as a 2-D matrix.
        kernellists_defaun = [weight1, weight2, weight3, weight4, weight5, weight6, weight7, weight8, weight9, weight10, weight11, weight12, weight13, weight14,
                        weight15, weight16, weight17, weight18, weight19, weight20, weight19, weight18, weight17, weight16, weight15,  weight14, weight13, weight12, weight11, weight10, weight9, weight8,
                        weight7, weight6, weight5, weight4, weight3, weight2, weight1]
        # Create the kernel from the weights.
        fixedkernel_defaun = ee.Kernel.fixed(39, 39, kernellists_defaun, -20, -20, True)
        return fixedkernel_defaun

    def getAsset(self):
        self.exportToAsset(self.total_indirect_pressure, 'total_indirect_pressure_20' + self.year, 'flii2v2_ephemeral/total_indirect_pressure_20' + self.year)
        self.exportToAsset(self.defaun_pressure, 'total_longrange_pressure_20' + self.year, 'flii2v2_ephemeral/total_longrange_pressure_20' + self.year)
        self.exportToAsset(self.defaun_pressure, 'fpi_20' + self.year, 'flii2v3_fpi/fpi_20' + self.year)
        
    def flii_metric(self):
                
        ratio_0_1 = self.final_ratio.multiply(-1).add(1)
        raw_intact = ratio_0_1.add(self.total_pressure_raw)
        final_metric = ee.Image.constant(10).subtract(raw_intact.multiply(ee.Number(10).divide(ee.Number(3))))
        final_metric = final_metric.where(final_metric.lte(0),0)
        self.export2drive(final_metric.multiply(10000).toInt().unmask(-9999), 'flii2v6_20' + self.year, self.aoi)
        
    def export2drive(self, image, description, region):
        
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=description,
            region=region,
            scale=self.scale,
            crs=self.crs,
            maxPixels=1e13
        )
        # try:
        #     psy_conn = psycopg2.connect(
        #         host=host,
        #         port=port,
        #         database=dbname,
        #         user=user,
        #         password=passwd
        #     )
        #     cursor = psy_conn.cursor()
        #     sql_convert2shp = f"INSERT INTO public.gee_collections (layer, region, status, start_date, end_date, filename, created) VALUES ('{MODIS_BURNT_AREA}', '{_region}', 'PROCESSED', '{self.start_date}', '{self.end_date}', '{_filename}', '{datetime.now()}')"
        #     cursor.execute(sql_convert2shp)
        #     psy_conn.commit()
            
        # except psycopg2.ProgrammingError as err:
        #     psy_conn.rollback()
        #     logging.error(f"Failed to insert record of {_filename}: {err}")
        
        # finally:
            # release the connection back to the pool
            # psy_conn.close()
            
        task.start()
        while task.active():
            print(f"Waiting on (id: {task.id})")
            time.sleep(30)
            
def main():
    args = _parse_args(sys.argv[1:])
    start = datetime.now()
    fetch_gee = FLII(year=args.year)
    fetch_gee.flii_metric()
    
    
if __name__ == "__main__":
    main()